# <h1 align="center"><font color="red">How to Build a Simple Reasoning and Acting Agent from Scratch</font></h1>

<font color="yellow">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este Notebook está baseado no Tutorial de [kirouane Ayoub]().

Links de estudo:

* [Part 1](https://blog.gopenai.com/introduction-to-llm-agents-how-to-build-a-simple-reasoning-and-acting-agent-from-scratch-part-1-843e14686be7)

* [Part 2](https://blog.gopenai.com/building-llm-agents-from-scratch-part-2-a-conversational-search-agent-with-ollama-a4544b2291cf)

# <font color="gree">Contextualizando</font>

Esta Notebook inicia uma série sobre a construção de `agentes de IA´, começando com uma implementação básica usando `Ollama´ para executar `LLMs´ localmente. O objetivo é criar um sistema que compreenda consultas do usuário e utilize pesquisa na web para fornecer respostas informativas, demonstrando os conceitos fundamentais do design de agentes de IA.

Esta implementação serve como um ponto de partida para entender o desenvolvimento de `agentes de IA`, mas é uma versão simplificada. Em cenários reais, os agentes teriam comportamentos mais complexos, exigindo técnicas de análise sintática robustas para lidar com saídas inesperadas. Além disso, seriam necessárias estratégias abrangentes de tratamento de erros para garantir uma operação suave e confiável do sistema.

# <font color="gree">Funcionamento do Agent</font>

<img src="https://miro.medium.com/v2/resize:fit:828/format:webp/1*-c8XfC8zGlrmF8DMwWkJaA.png" alt="Minha Imagem" width="700" height="400"/>


* `Agent:` Representa o sistema geral que abrange o LLM e suas interações com o ambiente.

* `Task:` Este é o objetivo ou meta que o agente está tentando atingir. É o ponto de partida do processo.

* `LLM:` Este é o núcleo do agente, um LLM que processa informações e toma decisões.

* `Reasoning:` O LLM utiliza o raciocínio para determinar o melhor curso de ação com base na tarefa e nas informações disponíveis.

* `Tools:` O agente pode utilizar várias ferramentas (`como pesquisa na web`, `calculadoras` ou `bancos de dados`) para coletar informações adicionais ou executar ações.

* `Action:` Com base em seu raciocínio, o LLM decide uma ação a ser tomada em seu ambiente.

* `Environment:` Representa o mundo externo ou contexto no qual o agente opera. `Pode ser um site`, `um espaço físico` ou `um ambiente simulado`.

* `Result:` A ação tomada pelo agente produz um resultado no ambiente, que então realimenta a compreensão e as ações futuras do agente.


# <font color="gree">Exemplo de código</font>

## <font color="yellow">Configurando Ollama e Dependências</font>

```
curl -fsSL https://ollama.com/install.sh | sh

ollama serve
ollama pull llama3.1
```

Também:

```
pip install openai duckduckgo-search
```

## <font color="yellow">Criando um cliente OpenAI para interação LLM local</font>

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
Eddy_key_openai  = os.environ['OPENAI_API_KEY']

from openai import OpenAI
# Inicializa o cliente OpenAI com a URL base do Ollama:
client = OpenAI(api_key=Eddy_key_openai,
                base_url="http://127.0.0.1:11434/v1"
                )

## <font color="yellow">Implementando a funcionalidade de pesquisa</font>

Nosso agente precisa da capacidade de pesquisar informações na `web`. Definimos uma função `search` que utiliza a biblioteca `duckduckgo-search` para consultar `DuckDuckGo` e recuperar resultados de pesquisa relevantes.



`Esses resultados serão usados ​​para aumentar o conhecimento do LLM quando necessário.`

In [2]:
from duckduckgo_search import DDGS

def search(query, num_results=2):
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=num_results))
    
    formatted_results = []
    for result in results:
        formatted_results.append(f"Título: {result['title']}\nConteúdo: {result['body']}\nURL: {result['href']}\n")
    
    return "\n".join(formatted_results) if formatted_results else "Nenhum resultado encontrado."

In [ ]:
# Testando o Duck:
from duckduckgo_search import DDGS

# Inicializa a pesquisa:
with DDGS() as ddgs:
    # Define a consulta de pesquisa:
    query = "Who won the 2024 Nobel Prize in Physics?"
    
    # Realiza a pesquisa e limita os resultados a 2:
    results = [r for r in ddgs.text(query, max_results=2)]
    
    # Exibe os resultados
    for result in results:
        print(result)

In [ ]:
def test_search():
    query = "Prêmio Nobel de Física 2024"
    results = search(query)
    print(f"Resultados da pesquisa para '{query}':")
    print(results)

# Execute esta função antes de rodar o agente:
test_search()

## <font color="yellow">Elaborando o Prompt do Sistema</font>

O prompt do sistema (`system prompt`) fornece instruções ao `LLM` sobre como se comportar. Ele descreve as ferramentas disponíveis (`neste caso, pesquisa na web`) e o processo de tomada de decisão para escolher entre respostas diretas e utilizar a ferramenta de pesquisa.


`Este prompt garante que o agente siga um fluxo de trabalho lógico.`

In [8]:
system_prompt = """
Você é um assistente AI avançado com acesso a pesquisa na internet em tempo real e um vasto conhecimento base. Siga estas instruções rigorosamente:

1. Para qualquer pergunta, use seu conhecimento base e os resultados de pesquisas anteriores. Se precisar de mais informações, inicie com "PESQUISA:" seguido por uma consulta clara e concisa.

2. NUNCA se recuse a fornecer informações ou se desculpe por não poder responder. Em vez disso, sempre forneça uma resposta útil e informativa.

3. Se a pergunta for sobre um evento futuro ou informação não disponível:
   a) Explique por que a informação específica não está disponível (por exemplo, "é um evento futuro").
   b) Forneça o contexto mais relevante e atualizado possível sobre o tópico.
   c) Mencione informações relacionadas, como datas esperadas de anúncio ou informações do ano anterior.
   d) Se apropriado, sugira onde o usuário pode encontrar atualizações no futuro.

4. Combine seu conhecimento base com as informações mais recentes encontradas nas pesquisas para dar a resposta mais completa possível.

5. Cite suas fontes, incluindo URLs, quando usar informações da pesquisa.

6. Mantenha suas respostas concisas, informativas e factuais.

Lembre-se: seu objetivo é sempre fornecer uma resposta útil e informativa, mesmo quando a informação específica solicitada não está disponível ou é sobre o futuro.
"""


## <font color="yellow">Construindo o Loop de Execução do Agente</font>

O núcleo (`core`) do nosso agente está na função `run_agent`. Esta função gerencia o loop de interação entre o `usuário`, o `LLM` e a `ferramenta de busca`.


`Ele processa os prompts do usuário, os envia ao LLM, interpreta a resposta do LLM, realiza pesquisas quando necessário e, por fim, fornece uma resposta final ao usuário.`

In [14]:
def run_agent(prompt, system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    max_iterations = 2
    previous_searches = set()

    for i in range(max_iterations):
        response = client.chat.completions.create(
            model="llama3.1:8b",
            messages=messages,
            temperature=0.3,
        )
        
        assistant_message = response.choices[0].message.content
        
        if assistant_message.startswith("PESQUISA:"):
            search_query = assistant_message.split("PESQUISA:")[1].strip().split('\n')[0]
            if search_query not in previous_searches:
                previous_searches.add(search_query)
                search_results = search(search_query)
                messages.append({"role": "assistant", "content": f"Realizando pesquisa: {search_query}"})
                messages.append({"role": "user", "content": f"Resultados da pesquisa:\n{search_results}\nCom base nesses resultados e no seu conhecimento base, responda à pergunta original de forma concisa e direta. Se a informação específica não estiver disponível, forneça o contexto mais relevante e atualizado que puder."})
            else:
                messages.append({"role": "assistant", "content": "Já realizei esta pesquisa. Vou responder com as informações disponíveis."})
        else:
            if "não está disponível" in assistant_message.lower() or "não foram encontradas" in assistant_message.lower():
                messages.append({"role": "user", "content": "Mesmo que a informação específica não esteja disponível, por favor, forneça o contexto mais relevante e atualizado que você puder sobre o tópico, combinando seu conhecimento base com os resultados da pesquisa."})
            else:
                return assistant_message

    # Resposta final
    final_response = client.chat.completions.create(
        model="llama3.1:8b",
        messages=messages,
        temperature=0.3,
    )
    return final_response.choices[0].message.content



## <font color="yellow">Exemplos ilustrativos</font>

Por fim, demonstramos as capacidades do `agente` com dois exemplos.

O primeiro mostra uma interação de conversação simples em que o agente responde diretamente a uma saudação.

In [ ]:
response = run_agent(prompt="Quem foi Albert Einstein?", system_prompt=system_prompt)

print(response)

In [ ]:
response = run_agent(prompt="Quem foram os ganhadores do Prêmio Nobel de Física de 2024 e quais foram as suas contribuções?", system_prompt=system_prompt)

print(response)

In [53]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
Eddy_key_openai  = os.environ['OPENAI_API_KEY']

from openai import OpenAI
# Inicializa o cliente OpenAI com a URL base do Ollama:
client = OpenAI(api_key=Eddy_key_openai,
                base_url="http://127.0.0.1:11434/v1"
                )

In [54]:
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

def search(query, max_results=1):
    # Realiza a pesquisa usando DuckDuckGo
    results = DDGS().text(query, max_results=max_results)
    
    # Verifica se há resultados
    if results:
        # Obtém a URL do primeiro resultado
        url = results[0].get('href')
        
        # Faz uma requisição para obter o HTML da página
        try:
            response = requests.get(url)
            response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
            
            # Usa BeautifulSoup para extrair texto relevante
            soup = BeautifulSoup(response.text, 'html.parser')
            text_content = soup.get_text(separator=' ', strip=True)  # Extrai todo o texto
            
            return text_content  # Retorna o texto completo extraído
        except requests.RequestException as e:
            return f"Erro ao buscar a página: {e}"
    
    return "Nenhum resultado encontrado."

In [65]:
query = "Quem foram os ganhadores do Prêmio Nobel de Física de 2024?"
html_content = search(query)

print(html_content)  # Verifique se o HTML está sendo retornado corretamente

Nobel de Física 2024 premia descoberta que impulsiona a inteligência artificial Notícias Notícias Política Fortaleza Economia Loteria Brasil Enem Ceará Mundo Saúde Tecnologia Ciência Curiosidades Veículos Checagem Publicações Legais Ao Vivo Previsão do Tempo Cariri Esportes Esportes Ceará SC Fortaleza EC Cearenses Futebol Jogos de Hoje Brasileirão Série A Brasileirão Série B Mais esportes Futebol Internacional Apostas Olimpíadas 2024 Paralimpíadas Mais times: Atlético-MG Botafogo Corinthians Cruzeiro Flamengo Fluminense Grêmio Internacional Palmeiras Santos São Paulo Vasco Divirta-se Divirta-se Signos A Fazenda Quiz BBB Masterchef Fica a Dica Vida & Arte Vida & Arte Agenda Cinema Guia Vida&Arte Turismo Shows & Espetáculos Exposições & Cursos Assine Os cookies nos ajudam a administrar este site. Ao usar nosso site, você concorda com nosso uso de cookies. Leia Mais or Aceitar . Nobel de Física 2024 premia descoberta que impulsiona a inteligência artificial Início Notícias Ciência Nobel d

In [67]:
import ast

def test_search():
    query = "Quem ganharam o Prêmio Nobel de Física 2024?"
    results = search(query)
    #print(f"Resultados da pesquisa para '{query}':")
    # results = ast.literal_eval(results)
    # results = results["body"]
    print(results)

# Execute esta função antes de rodar o agente:
test_search()

Nobel de Física 2024 premia descoberta que impulsiona a inteligência artificial Notícias Notícias Política Fortaleza Economia Loteria Brasil Enem Ceará Mundo Saúde Tecnologia Ciência Curiosidades Veículos Checagem Publicações Legais Ao Vivo Previsão do Tempo Cariri Esportes Esportes Ceará SC Fortaleza EC Cearenses Futebol Jogos de Hoje Brasileirão Série A Brasileirão Série B Mais esportes Futebol Internacional Apostas Olimpíadas 2024 Paralimpíadas Mais times: Atlético-MG Botafogo Corinthians Cruzeiro Flamengo Fluminense Grêmio Internacional Palmeiras Santos São Paulo Vasco Divirta-se Divirta-se Signos A Fazenda Quiz BBB Masterchef Fica a Dica Vida & Arte Vida & Arte Agenda Cinema Guia Vida&Arte Turismo Shows & Espetáculos Exposições & Cursos Assine Os cookies nos ajudam a administrar este site. Ao usar nosso site, você concorda com nosso uso de cookies. Leia Mais or Aceitar . Nobel de Física 2024 premia descoberta que impulsiona a inteligência artificial Início Notícias Ciência Nobel d

In [56]:
import requests
from bs4 import BeautifulSoup

import requests

def fetch_and_pass_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
        
        # Retorna o conteúdo HTML completo
        return response.text
    
    except Exception as e:
        return f"Erro ao buscar a página: {e}"

In [57]:
system_prompt = """
Você é um assistente inteligente que responde perguntas do usuário.
Se você não souber a resposta, responda com <não sei> sempre.
Se você não souber a resposta (se responder com <não sei>) você deverá pesquisar na Web através da 
função implementada (<search>) e seguidamente responder ao usuário com apenas um PARÁGRAFO RESUMIDO. 
Sempre busque fornecer informações precisas e relevantes.
Limite a sua resposta em 500 caracteres como máximo.
"""


In [32]:
def run_agent(prompt, system_prompt):
    # Mensagens iniciais
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    previous_searches = set()  # Para evitar consultas repetidas

    while True:
        response = client.chat.completions.create(
            model="llama3.1:8b",
            messages=messages,
        )

        answer = response.choices[0].message.content
        
        # Verifica se a resposta é satisfatória
        if "não sei" in answer.lower() or "pesquisar" in answer.lower():
            query = prompt  # Pode ser refinado conforme necessário
            if query not in previous_searches:
                previous_searches.add(query)
                search_results = search(query)
                
                return search_results  # Retorna os resultados da pesquisa
        else:
            return answer  # Retorna a resposta do modelo


In [106]:
def run_agent(prompt, system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    previous_searches = set()

    while True:
        response = client.chat.completions.create(
            model="llama3.1:8b",
            messages=messages,
        )

        answer = response.choices[0].message.content
        
        if "não sei" in answer.lower() or "pesquisar" in answer.lower():
            query = prompt
            if query not in previous_searches:
                previous_searches.add(query)
                text_content = search(query)
                print(text_content)
                messages = [
                            {"role": "system", "content": """Você é um assistente que resume o seguinte texto em apenas 500 caracteres"""},
                            {"role": "user", "content": text_content}
                        ]

                # Refaça a chamada ao modelo com o texto extraído:
                response_with_text = client.chat.completions.create(
                    model="llama3.1:8b",
                    messages=messages,
                )
                
                return response_with_text.choices[0].message.content  # Retorna a resposta do modelo
            
            return "Nenhum resultado encontrado."
        else:
            return answer


In [107]:
# Exemplo 1: Pergunta conhecida
response1 = run_agent(prompt="Quem foi Albert Einstein", system_prompt=system_prompt)
print(response1)

Albert Einstein (1879-1955) foi um físico teórico alemão-brasileiro considerado uma das maiores inteligências do século XX. Ele desenvolveu a Teoria da Relatividade, que revolucionou a compreensão do espaço e do tempo, e foi laureado com o Prêmio Nobel de Física em 1921.


In [109]:
# Exemplo 2: Pergunta específica que requer pesquisa
response2 = run_agent(prompt="Quem foram os ganhadores do Prêmio Nobel de Física de 2024?", system_prompt=system_prompt)

print(response2)

Nobel de Física 2024 premia descoberta que impulsiona a inteligência artificial Notícias Notícias Política Fortaleza Economia Loteria Brasil Enem Ceará Mundo Saúde Tecnologia Ciência Curiosidades Veículos Checagem Publicações Legais Ao Vivo Previsão do Tempo Cariri Esportes Esportes Ceará SC Fortaleza EC Cearenses Futebol Jogos de Hoje Brasileirão Série A Brasileirão Série B Mais esportes Futebol Internacional Apostas Olimpíadas 2024 Paralimpíadas Mais times: Atlético-MG Botafogo Corinthians Cruzeiro Flamengo Fluminense Grêmio Internacional Palmeiras Santos São Paulo Vasco Divirta-se Divirta-se Signos A Fazenda Quiz BBB Masterchef Fica a Dica Vida & Arte Vida & Arte Agenda Cinema Guia Vida&Arte Turismo Shows & Espetáculos Exposições & Cursos Assine Os cookies nos ajudam a administrar este site. Ao usar nosso site, você concorda com nosso uso de cookies. Leia Mais or Aceitar . Nobel de Física 2024 premia descoberta que impulsiona a inteligência artificial Início Notícias Ciência Nobel d